In [1]:
import nltk
import gensim
import numpy as np
import keras.backend as K

from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# #Pretrained Model - Word Embedding - Word2Vec
# from nltk.data import find
# word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
# word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)


# Load Google's pre-trained Word2Vec model.
word_embedding_model = gensim.models.KeyedVectors.load_word2vec_format('W2V_Model/GoogleNews-vectors-negative300.bin/data', binary=True)

C:\Users\ZEESHAN\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\ZEESHAN\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\ZEESHAN\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Importing Train and Test Data
import csv

emotions = ['anger', 
            # 'anticipation', 
            'disgust', 
            'fear', 
            'joy', 
            # 'love',
            'optimism', 
            'pessimism', 
            'sadness', 
            'surprise'] #, 
            # 'trust']

x_train_raw = []
y_train_raw = []
x_test_raw = []
y_test_raw = []


with open('Data/train.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        x_train_raw.append(row['Tweet'])
        y_train_raw_temp = []
        for emotion in emotions:
            y_train_raw_temp.append(row[emotion])
        y_train_raw.append(y_train_raw_temp)
        
with open('Data/test.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        x_test_raw.append(row['Tweet'])
        y_test_raw_temp = []
        for emotion in emotions:
            y_test_raw_temp.append(row[emotion])
        y_test_raw.append(y_test_raw_temp)
        
train_size = len(y_train_raw)
test_size = len(y_test_raw)

print("Train Size:", train_size, " samples")
print("Test Size:", test_size, " samples")

Train Size: 6838  samples
Test Size: 886  samples


In [3]:
SentiWords = {}
with open('Data/words.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        w = row['Words']
        SentiWords_temp = np.zeros(len(emotions), dtype=K.floatx())
        for i, emotion in enumerate(emotions):
            SentiWords_temp[i] = row[emotion]
        SentiWords[w] = SentiWords_temp
    SentiWords["QQ"] = np.zeros(len(emotions), dtype=K.floatx())

In [4]:
# initialization Block
max_n_words = 40
ignore_words = ['?', '@', '-', '.', '_', '/', ' ', '.', '!',
               "you'll", 'itself', 'some', 'same', 'off', 'any', 'having',
                'and', 'theirs', 'your', 'should', 'after', 'out', 'in', 
                "you'd", 'd', 'its', 'had', 'myself','from', 'ourselves', 
                'here', 'an', 'all', 'yours', 'as', 'hers', 'they', 'll',
                "she's", 'through', 'you', 'then', 'once', 'my', 'am', 'who',
                'being', 'of', 'shan', 'that', 'so', 'with', 'yourselves',
                'both', 't', 'his', 'we', 'more', 'did', 'our', 'he', 'o', 
                'them', 'than', 'it', 'y', 'her', 'up', 'about', 'this', 
                'himself', 'just', 'if', 'own', 'has', 'how', 'because', 
                'him', 'doing', 'at', 'm', 'is', 'each', 's', 'too', 'those', 
                'such', 'have', 'above', "you've", 'most', 'on', 'under', 
                'by', 'few', 'where', 'when', 'were', "you're", "it's",
                'been', 'the', 'before', 'do', 'these', 'other', 'to', 'i',
                'can', 'themselves', 'what', 'are', 'while', 'which', 'me',
                'ma', "that'll", 've', 'for', 'why', 'a', 'during', 'yourself',
                'below', 'now', 'only', 'their', 'herself', 'will', 'does', 
                'she', 'be', 'there', "should've", 'was', 're', 'ours', 
                'whom', 'further']

# ignore_words = ['?', '@', '-', '.', '_', '/', ' ', '.', '!']

vector_size = len(word_embedding_model['I'])

print ("Vocabulary size: ",len(word_embedding_model.vocab), " words")

print ("Word-Vector size used: ", vector_size)

Vocabulary size:  3000000  words
Word-Vector size used:  300


In [5]:
# Cleaning and vectorizing the data
max_n_words=40

vector_size = len(word_embedding_model['I'])
# Lemmatizing Functions
def get_wordnet_pos(tag):
    if (tag == ''):
        return ''
    elif tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('V'):
        return wn.VERB
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    else:
        return ''

def lemmatize_w(word, tag):
    wn_tag = get_wordnet_pos(tag)
    if (wn_tag == ''):
        return word
    else:
        return WordNetLemmatizer().lemmatize(word, wn_tag)

def lemmatize_s(sentence):
#     sent = []
#     word_tag = nltk.pos_tag(sentence)
    
#     for w, t in word_tag:
#         sent.append(lemmatize_w(w, t))
#     return sent
    return sentence


# Filtering the data
def filter_sent (sentence, embedding_model = word_embedding_model):
    
    # tokenize each word in the sentence
    s_words = word_tokenize(sentence.lower())
    
    filtered_sentence = []
    
    l = lemmatize_s(s_words)
    
    for w in l:
        # Remove words not in Vocab
        if w in embedding_model.vocab:
            filtered_sentence.append(w)
            
    return filtered_sentence

# Converting data to vectors
x_train = np.zeros((train_size, max_n_words, vector_size + len(emotions)), dtype=K.floatx())
y_train = np.zeros((train_size, len(emotions)), dtype=np.int32)
x_test = np.zeros((test_size, max_n_words, vector_size + len(emotions)), dtype=K.floatx())
y_test = np.zeros((test_size, len(emotions)), dtype=np.int32)

for i in range(train_size):
    x = filter_sent(x_train_raw[i])
    for index, word in enumerate(x):
        if word in SentiWords:
            vec = SentiWords[word]
        else:
            vec = SentiWords["QQ"]
        x_train[i, index, :] = np.concatenate((word_embedding_model[word], vec), axis=0)
        
for i, y in enumerate(y_train_raw):
    y_temp = np.zeros(len(emotions), dtype=np.int32)
    y_temp = np.array(y, dtype=np.int32)
    y_train[i, :] = y_temp
        
for i in range(test_size):
    x = filter_sent(x_test_raw[i])
    for index, word in enumerate(x):
        if word in SentiWords:
            vec = SentiWords[word]
        else:
            vec = SentiWords["QQ"]
        x_test[i, index, :] = np.concatenate((word_embedding_model[word], vec), axis=0)

        
for i, y in enumerate(y_test_raw):
    y_temp = np.zeros(len(emotions), dtype=np.int32)
    y_temp = np.array(y, dtype=np.int32)
    y_test[i, :] = y_temp
    
print ("Data Vectorized")

print ("Input shape: ", x_train.shape)
print ("Output shape: ", y_train.shape)

Data Vectorized
Input shape:  (6838, 40, 308)
Output shape:  (6838, 8)


In [12]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing import sequence
from keras.layers import LSTM, Embedding

# Keras model
batch_size = 32  # 10
nb_epochs = 10   # 20

model = Sequential()

# CNN
model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same', 
                 input_shape=(max_n_words, vector_size + len(emotions))))
model.add(Dropout(0.5))
model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(2))

model.add(Conv1D(32, kernel_size=2, activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(Conv1D(32, kernel_size=2, activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(2))

model.add(Flatten())

model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))  # tanh # relu
model.add(Dropout(0.25))

model.add(Dense(len(emotions), activation='sigmoid'))  # softmax # 

# Compile the model
model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1e-7),
              metrics = ['accuracy'])

print("MODEL:")
print(model.summary(), "\n")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
MODEL:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 40, 64)            59200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 40, 64)            12352     
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 64)            0         
_________________________________________________________________
conv1d_3 (Conv

In [13]:
# Fit the model
#x_train = np.zeros((train_size, max_n_words, vector_size + len(emotions)), dtype=K.floatx())
#y_train = np.zeros((train_size, len(emotions)), dtype=np.int32)
#x_test = np.zeros((test_size, max_n_words, vector_size + len(emotions)), dtype=K.floatx())
#y_test = np.zeros((test_size, len(emotions)), dtype=np.int32)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    shuffle=False, #False,
                    epochs=nb_epochs,
                    verbose=2,
                    validation_data=(x_test, y_test),
                    callbacks=[EarlyStopping(min_delta=1e-7, patience=3)])  # min_delta=0.00025, patience=2

# Fit the model (without early stop)
# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     shuffle=False, #False
#                     epochs=nb_epochs,
#                     verbose=2,
#                     validation_data=(x_test, y_test))

print ("\n================================== Model Trained =================================\n")

Instructions for updating:
Use tf.cast instead.
Train on 6838 samples, validate on 886 samples
Epoch 1/10
 - 11s - loss: 0.5330 - acc: 0.7414 - val_loss: 0.4605 - val_acc: 0.8081
Epoch 2/10
 - 6s - loss: 0.4648 - acc: 0.7907 - val_loss: 0.4273 - val_acc: 0.8200
Epoch 3/10
 - 6s - loss: 0.4424 - acc: 0.8042 - val_loss: 0.4138 - val_acc: 0.8239
Epoch 4/10
 - 6s - loss: 0.4293 - acc: 0.8126 - val_loss: 0.4103 - val_acc: 0.8238
Epoch 5/10
 - 6s - loss: 0.4188 - acc: 0.8183 - val_loss: 0.4080 - val_acc: 0.8232
Epoch 6/10
 - 6s - loss: 0.4128 - acc: 0.8192 - val_loss: 0.4050 - val_acc: 0.8244
Epoch 7/10
 - 6s - loss: 0.4013 - acc: 0.8253 - val_loss: 0.4038 - val_acc: 0.8207
Epoch 8/10
 - 6s - loss: 0.3911 - acc: 0.8308 - val_loss: 0.3982 - val_acc: 0.8235
Epoch 9/10
 - 6s - loss: 0.3845 - acc: 0.8333 - val_loss: 0.3957 - val_acc: 0.8284
Epoch 10/10
 - 6s - loss: 0.3785 - acc: 0.8376 - val_loss: 0.3927 - val_acc: 0.8249

================================== Model Trained =======================

In [14]:
# Plotting

import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [15]:
# Classification
ERROR_THRESHOLD = 0.40

def classify(sentence, model, show_details=False):
    
    inp = np.zeros((train_size, max_n_words, vector_size + len(emotions)), dtype=K.floatx())
    
    filtered_sentence = filter_sent(sentence)
    x = filtered_sentence
        
    for index, word in enumerate(x):
        if word in SentiWords:
            vec = SentiWords[word]
        else:
            vec = SentiWords["QQ"]
        inp[0, index, :] = np.concatenate((word_embedding_model[word], vec), axis=0)
    
    results = model.predict(x = inp, batch_size=None)
    
    results2 = [[i,r] for i,r in enumerate(results[0]) if r > ERROR_THRESHOLD ] 
    results2.sort(key=lambda x: x[1], reverse=True)
    return_results = [[emotions[r[0]],r[1]] for r in results2]
    
    if show_details:
        print (emotions)
        print (results[0])
    
    if len(return_results) == 0:
        return_results.append("Neutral")
        
    return return_results

In [16]:
# Manual Testing
test_x = "He is so bad"
print (test_x, "\n")
print ("Prediction: ", classify(test_x, model), "\n")

He is so bad 

Prediction:  [['sadness', 0.50519097], ['disgust', 0.4918079], ['anger', 0.40325317]] 



In [17]:
# Random manual Testing
n = np.random.randint(test_size - 1)
test_x = x_test_raw[n]

# n = np.random.randint(train_size - 1)
# test_x = x_train_raw[n]

print (test_x,"\n")

tags = []
for i, val in enumerate(y_test_raw[n]):
    if val == '1':
        tags.append(emotions[i])
print ("Actual Tag: ", tags, "\n")

# for i, val in enumerate(y_train_raw[n]):
#     if val == '1':
#         tags.append(emotions[i])
# print ("Actual Tag: ", tags)

print ("Prediction: ", classify(test_x, model), "\n")

Even in an incomplete state , the ten-tails possesses immense physical strength and is remarkably fast , despite its tremendous size 

Actual Tag:  ['joy', 'optimism', 'surprise'] 

Prediction:  [['joy', 0.91564876], ['optimism', 0.7865144]] 



In [18]:
# Saving Model
# serialize model to JSON
model_json = model.to_json()
with open("Models/model_CNN_miltilabel.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("Models/model_CNN_miltilabel.h5")
print ("Saved model to disk")

Saved model to disk


In [19]:
# Loading Model

from keras.models import model_from_json

# load json and create model
json_file = open('Models/model_CNN_miltilabel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("Models/model_CNN_miltilabel.h5")
print ("Loaded model from disk")

Loaded model from disk


In [20]:
# Evaluate loaded model on test data
loaded_model.compile(loss = 'binary_crossentropy', 
                     optimizer = RMSprop(lr = 0.002, rho = 0.9, epsilon = None, decay = 1e-7),
                     metrics=['accuracy'])
score = loaded_model.evaluate(x_test, y_test, verbose=1)
print ("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

886/886 [==============================] - 0s 498us/step
acc: 82.49%


In [21]:
# Manual Testing
test_x = input("Enter the text in English: ")
print (test_x, "\n")
print ("Prediction: ", classify(test_x, loaded_model),"\n")

Enter the text in English: Hello brother.. how are u???
Hello brother.. how are u??? 

Prediction:  ['Neutral'] 

